# Exercise Notebook on Interacting with Databases

In [1]:
import pandas as pd
import numpy as np
import os.path

In this exercise notebook we will work with a SQLite database. First, let's create a new database from the *movielens* dataset. The ratings data is very large and ingesting it into SQLite is quite slow. Let's only take the first 100 thousand records.

We use the Pandas `.readcsv(filepath, sep)` method to load in the csv data. We join on the file name using the `os.path.join(path, paths)`method to create the full path name.

In [3]:
# DO NOT MODIFY

# set here the relative path to the movielens folder
MOVIELENS="../Week-4-Pandas/movielens/"

movies = pd.read_csv(os.path.join(MOVIELENS, 'movies.csv'), sep=',')
ratings = pd.read_csv(os.path.join(MOVIELENS, 'ratings.csv'), sep=',')

Using the Pandas .iloc property we can extract the data we want (first 100 thousand data samples)

In [4]:
# DO NOT MODIFY

ratings = ratings.iloc[:100000]

We first create on disk the SQLite databse `movielens.db`. Using sqlite3's `.connect(database)` method creates a new database if the name doesn't exist in the path, otherwise it will open a connection to the existing database. 

In [5]:
# DO NOT MODIFY

import sqlite3
conn = sqlite3.connect("movielens.db")

Using the `os.path.getsize(filename)` method, we return the size of the argued file. We also use string formatting, with the type as 'g', indicating general format. 

Specifying `:.2g` indicates we want 2 d.p. in this case (other cases will vary using 'g'.

In [6]:
print("Size of the database is {:.2g} MB".format(os.path.getsize("movielens.db")/1024**2))

Size of the database is 0 MB


We use the `.to_sql(name, con, if_exists)` method of Pandas DataFrames to store the data there. We specify that if the table exists already, it should be replaced.

In [7]:
# DO NOT MODIFY
movies.to_sql("movies", conn, if_exists="replace")

In [8]:
# DO NOT MODIFY
ratings.to_sql("ratings", conn, if_exists="replace")

Note: We divide the number returned from `os.path.getsize(filename)` by 1024^2 as it returns the number of bytes of a file (and we want MB).

In [26]:
print("Size of the database is {:.2g} MB".format(os.path.getsize("movielens.db")/1024**2))

Size of the database is 5.5 MB


## Exercise 1: get the names of all the tables in the database 

First let's find out the name of the tables available inside the database

In [45]:
cursor = conn.cursor()

The sqlite.Cursor `.fetchall()` method returns a list of tuples. To return these tuples as strings in a list, we can iterate through them and return in list comprehension by indexing by [0] for each tuple.

In [46]:
def get_list_of_tables(sqlite_cursor):
    """Return a list of table names in a sqlite database
    
    Parameters
    ----------
    sqlite_cursor : sqlite3.Cursor
    SQLite database cursor
    
    Returns
    -------
    tables : list of strings
    list of table names"""
### BEGIN SOLUTION
    x =  sqlite_cursor.execute("SELECT name FROM sqlite_master").fetchall()
    x = [i[0] for i in x]
    return x
### END SOLUTION

In [47]:
movielens_db_tables = get_list_of_tables(cursor)
movielens_db_tables

['movies', 'ix_movies_index', 'ratings', 'ix_ratings_index']

The `isinstance(obj, class_or_tuple)` method returns whether an object is an instance of a class or of a subclass thereof. If the assert statement is False (e.g. the object is not an instance of the given class), it will pull up the assertion error.

In [48]:
# DO NOT MODIFY

assert isinstance(movielens_db_tables[0], str), "Make sure that the function does not return a list of tuples but just a list of strings"
assert "movies" in movielens_db_tables, "It looks like the movies table is missing"
assert "ratings" in movielens_db_tables, "It looks like the ratings table is missing"

## Exercise 2: Print a few ratings records

Let's use the `sqlite3` package to extract the first 3 ratings from the ratings table.

In [55]:
first_3_ratings = None
### BEGIN SOLUTION
first_3_ratings = cursor.execute("SELECT * from ratings LIMIT 3").fetchall()
### END SOLUTION

In [61]:
first_3_ratings

[(0, 1, 2, 3.5, 1112486027),
 (1, 1, 29, 3.5, 1112484676),
 (2, 1, 32, 3.5, 1112484819)]

In [57]:
assert len(first_3_ratings) == 3, "Make sure you are selecting only 3 rows"
assert len(first_3_ratings[0]) == 5, "Something went wrong, each row of ratings should have 5 columns"

Exploring the first_3_ratings data structure, we can see it is a Python list. Through indexing, we can see that it is a list of tuples.

In [62]:
print(type(first_3_ratings))
print(type(first_3_ratings[0]))

<class 'list'>
<class 'tuple'>


## Exercise 3: Load the ratings table in Pandas

Let's load the entire ratings table into a dataframe. We do this using Panda's `.read_sql_query(sql, con)` method.

In [68]:
ratings_dataframe = None
### BEGIN SOLUTION
ratings_dataframe = pd.read_sql_query("SELECT * from ratings", conn)
### END SOLUTION

In [69]:
assert len(ratings_dataframe) == 100000, "Check your code, you should have 100000 rows"

## Exercise 4: Load a subset of the movies table in Pandas

We would like now to only load the movies of the Fantasy genre into a Pandas dataframe, not the complete movies table. This is very useful for large datasets because we can save a lot of memory if you avoid doing the filtering in pandas but let the database take care of it.

Let's first introduce the SQL syntax for checking if a records contains a string, for example let's print the first 3 Children movies:

In [18]:
cursor.execute("SELECT * FROM movies WHERE genres LIKE '%Children%' LIMIT 3").fetchall()

[(0, 1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (1, 2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (7, 8, 'Tom and Huck (1995)', 'Adventure|Children')]

In [77]:
fantasy_movies_dataframe = None
### BEGIN SOLUTION
fantasy_movies_dataframe = pd.read_sql_query("SELECT * FROM movies WHERE genres LIKE '%Fantasy%'", conn)
### END SOLUTION
fantasy_movies_dataframe

,index,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,28,29,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
3,43,44,Mortal Kombat (1995),Action|Adventure|Fantasy
4,55,56,Kids of the Round Table (1995),Adventure|Children|Fantasy
5,59,60,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy
6,124,126,"NeverEnding Story III, The (1994)",Adventure|Children|Fantasy
7,186,188,"Prophecy, The (1995)",Fantasy|Horror|Mystery
8,187,189,Reckless (1995),Comedy|Fantasy
9,199,201,Three Wishes (1995),Drama|Fantasy


In our second assert statement below, we first index by genres in our fantasy_movies_dataframe to convert it to a series. We then use the Pandas vectorized string method `.str.contains()` to return a boolean Index based on whether a given pattern or regex is contained within a string of a Series or Index. Finally, the DataFrame `.all()` method at the end returns True if all elements are True, otherwise returns False.

In [80]:
# DO NOT MODIFY

assert len(fantasy_movies_dataframe) == 1412, "Make sure you do not have LIMIT in the SQL query"
assert fantasy_movies_dataframe.genres.str.contains("Fantasy").all(), "All rows should contain the Fantasy string, try to identify which rows are not"